In [1]:
#adicione o patching aqui Exemplo: 
#patch = "/kaggle/input/grafo1/Grafo1.xlsx"

patch = "Adicione o patching aqui"

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# Espero que este notebook funcione para poupar um pouco da minha sanidade mental que me resta.
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime # convert to datetime format

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
tabela = pd.read_excel(patch)

tabela['Emissor'] = tabela['Emissor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
tabela['Receptor'] = tabela['Receptor'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

tabela['Emissor'] = tabela['Emissor'].str.title()
tabela['Receptor'] = tabela['Receptor'].str.title()

tabela['Emissor'] = tabela['Emissor'].replace([''],['?'])
tabela['Receptor'] = tabela['Receptor'].replace([''],['?'])

start_time = pd.to_datetime(tabela['Tempo'], format='%H:%M:%S')

tabela.Tempo_Fim = tabela['Tempo_Fim'].replace([0],['0:00:00'])

final_time = pd.to_datetime(tabela['Tempo_Fim'], format='%H:%M:%S')

time_tabel = pd.concat([start_time, final_time], axis = 1)

time_tabel['Total_Time'] = time_tabel.Tempo_Fim - time_tabel.Tempo

time_tabel['Total_Seconds'] = time_tabel['Total_Time'].dt.total_seconds()

time_tabel['Total_Reajuste'] = time_tabel['Total_Seconds'].apply(lambda x: 1 if x <= 0 else x + 1)

Converter_Grafo = pd.concat([time_tabel['Total_Reajuste'], tabela['Emissor'], tabela['Receptor']], axis = 1)

Converter_Grafo = Converter_Grafo.dropna()

Converter_Grafo2 = Converter_Grafo.loc[Converter_Grafo.index.repeat(Converter_Grafo.Total_Reajuste)].reset_index(drop=True)

Convertido = pd.concat([Converter_Grafo2['Emissor'], Converter_Grafo2['Receptor']], axis = 1)

Convertido_SemGrupo = Convertido[(Convertido["Receptor"].str.contains("Grupo") == False)].reset_index(drop=True)
Convertido_SemGrupo = Convertido_SemGrupo[(Convertido_SemGrupo["Emissor"].str.contains("Emissor") == False)].reset_index(drop=True)

/kaggle/input/grafo1/Grafo1.xlsx


In [3]:
Convertido.to_excel("Grafo.xlsx")

In [4]:
Convertido_SemGrupo.to_excel("Grafo_SemGrupo.xlsx")